<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 11</p>
<h4>Names........Group-2</h4>
<ol>
    <li>Vijay Karthik Bethapudi</li>
    <li>Prakhar Patel</li>
    <li>Vishal Dharmendra Patel</li>
    <li>Vivek Patel</li>
    <li>Yash Suresh Upadhyay</li>
    <li>Archana Wasti Dahal</li>
    <li> Huzaifa Jahan</li>
    <li>Iretioluwa Olawuyi</li>
</ol>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [9]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  1.0  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  3.0


In [3]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.NaN
A[4][6] = np.NaN

A_df = pd.DataFrame(A)
print (A_df.head())

     0    1    2    3    4    5    6
0  4.0  2.0  4.0  5.0  4.0  2.0  2.0
1  4.0  4.0  3.0  2.0  2.0  5.0  4.0
2  4.0  1.0  4.0  7.0  3.0  6.0  2.0
3  8.0  NaN  2.0  0.0  5.0  0.0  7.0
4  4.0  5.0  8.0  7.0  6.0  2.0  NaN


In [4]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.
K = 1
P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [5]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    eij = Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eRating_Matrix = np.dot(P,Q)
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e = e + pow(Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        #print("Total error at step", step, "is", e)
        if e < 0.0001:
            break
    return P, Q.T

In [6]:
eP, eQ = matrix_factorization(A, P, Q.T, K, steps = 1000)
eA = np.matmul(eP, eQ.T)

print(A,'\n')
print(eA)

[[ 4.  2.  4.  5.  4.  2.  2.]
 [ 4.  4.  3.  2.  2.  5.  4.]
 [ 4.  1.  4.  7.  3.  6.  2.]
 [ 8. nan  2.  0.  5.  0.  7.]
 [ 4.  5.  8.  7.  6.  2. nan]] 

[[3.88450179 2.57075299 3.74773819 3.69103013 3.45556594 2.35975955
  3.55853632]
 [3.77147533 2.49595237 3.63869111 3.58363307 3.35502012 2.29109816
  3.4549944 ]
 [4.42075011 2.92564071 4.26510655 4.20057005 3.93260045 2.68552001
  4.04978571]
 [4.28027417 2.8326741  4.12957641 4.06709065 3.80763619 2.6001836
  3.92109771]
 [6.35441219 4.20533314 6.13068921 6.03792407 5.652743   3.86018224
  5.82118578]]


In [7]:
nan_Indices = np.argwhere(np.isnan(A))
total_Error = 0

for index in nan_Indices:
    i, j = index
    total_Error += abs(eA[i, j] - A_orig[i, j]) / A_orig[i, j]

# Average percentage error
ape = total_Error / len(nan_Indices)

print("Average percentage error : ", ape)

Average percentage error :  1.3865346780335785
